In [1]:
import pandas as pd
df = pd.read_csv('TrackingList.csv')

In [3]:
targets_df = df # [df.index > 100]
targets_df

,Tracking Nos.,Client Nos.,Comments
0,70191640000164106422,38525.02,4868 and check payment; Stack 1
1,70192280000060192481,48755.02,4868 and check payment; Stack 1
2,9114902200789659810412,23126.02,Stack 1
3,9114902200789659710238,2541.02,Source Docs; Stack 1
4,9114902200789659810320,94890.02,Stack 1
...,...,...,...
198,9114901230801247551423,54285.02,Source Docs
199,9114901230801247551430,54156.02,Source Docs
200,9114901230801247551447,64730.02,Source Docs
201,9114901230801247551454,77138.02,Source Docs


In [4]:
base_url = 'https://secure.shippingapis.com/ShippingAPI.dll?API=TrackV2&XML='
from config import userid, password

call_url = base_url + f'<TrackRequest USERID="{userid}" PASSWORD="{password}">'

batch_list = targets_df['Tracking Nos.']
for number in batch_list:
    call_url += f'<TrackID ID="{number}"></TrackID>'
call_url += '</TrackRequest>'


In [5]:
import requests
from bs4 import BeautifulSoup

response = requests.get(call_url)
soup = BeautifulSoup(response.content, 'xml')

In [6]:
soup

<?xml version="1.0" encoding="utf-8"?>
<TrackResponse><TrackInfo ID="70191640000164106422"><TrackSummary>Your package is moving within the USPS network and is on track to be delivered to its final destination. It is currently in transit to the next facility.</TrackSummary><TrackDetail>In Transit to Next Facility, 04/22/2024</TrackDetail><TrackDetail>In Transit to Next Facility, 04/21/2024</TrackDetail><TrackDetail>In Transit to Next Facility, 04/20/2024</TrackDetail><TrackDetail>Arrived at USPS Regional Facility, April 18, 2024, 8:36 am, CHARLOTTE NC DISTRIBUTION CENTER</TrackDetail><TrackDetail>In Transit to Next Facility, 04/17/2024</TrackDetail><TrackDetail>Arrived at USPS Regional Facility, April 16, 2024, 12:34 am, COPPELL TX DISTRIBUTION CENTER</TrackDetail></TrackInfo><TrackInfo ID="70192280000060192481"><TrackSummary>Your package is moving within the USPS network and is on track to be delivered to its final destination. It is currently in transit to the next facility.</TrackSum

In [7]:
string = str(soup)
string = string.replace('><','>\n<')
string = string.replace('</TrackInfo>','</TrackInfo>\n--\n')
string

'<?xml version="1.0" encoding="utf-8"?>\n<TrackResponse>\n<TrackInfo ID="70191640000164106422">\n<TrackSummary>Your package is moving within the USPS network and is on track to be delivered to its final destination. It is currently in transit to the next facility.</TrackSummary>\n<TrackDetail>In Transit to Next Facility, 04/22/2024</TrackDetail>\n<TrackDetail>In Transit to Next Facility, 04/21/2024</TrackDetail>\n<TrackDetail>In Transit to Next Facility, 04/20/2024</TrackDetail>\n<TrackDetail>Arrived at USPS Regional Facility, April 18, 2024, 8:36 am, CHARLOTTE NC DISTRIBUTION CENTER</TrackDetail>\n<TrackDetail>In Transit to Next Facility, 04/17/2024</TrackDetail>\n<TrackDetail>Arrived at USPS Regional Facility, April 16, 2024, 12:34 am, COPPELL TX DISTRIBUTION CENTER</TrackDetail>\n</TrackInfo>\n--\n\n<TrackInfo ID="70192280000060192481">\n<TrackSummary>Your package is moving within the USPS network and is on track to be delivered to its final destination. It is currently in transit t

In [8]:
file = open('report.txt','a')
file.write(string)
file.close()